In [ ]:
!pip install --q gradio

     |████████████████████████████████| 865 kB 3.6 MB/s 
     |████████████████████████████████| 2.0 MB 45.3 MB/s 
     |████████████████████████████████| 210 kB 54.0 MB/s 
     |████████████████████████████████| 61 kB 416 kB/s 
     |████████████████████████████████| 3.6 MB 48.6 MB/s 
     |████████████████████████████████| 856 kB 71.5 MB/s 


In [16]:
import torch
import torch.nn as nn
import gradio as gr
from PIL import Image
from torchvision import models, transforms
import pickle
import copy

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# example inputs
example_inp = []
!wget -O p_cresphontes.jpg https://www.baltana.com/files/wallpapers-6/Papilio-Cresphontes-Background-Wallpaper-20293.JPG
example_inp.append(["/p_cresphontes.jpg"])
!wget -O h_erato.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Red_postman_butterfly_%28Heliconius_erato%29.jpg/1599px-Red_postman_butterfly_%28Heliconius_erato%29.jpg
example_inp.append(["/h_erato.jpg"])
!wget -O d_plexippus.jpg https://www.ukbutterflies.co.uk/photo_album/source/664a285ca7b4379147d598ea5127228f.jpg
example_inp.append(["d_plexippus.jpg"])

--2022-01-15 09:55:25--  https://www.baltana.com/files/wallpapers-6/Papilio-Cresphontes-Background-Wallpaper-20293.JPG
Resolving www.baltana.com (www.baltana.com)... 78.159.100.204
Connecting to www.baltana.com (www.baltana.com)|78.159.100.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220984 (216K) [image/jpeg]
Saving to: ‘p_cresphontes.jpg’

p_cresphontes.jpg   100%[===================>] 215.80K  81.6KB/s    in 2.6s    

2022-01-15 09:55:29 (81.6 KB/s) - ‘p_cresphontes.jpg’ saved [220984/220984]

--2022-01-15 09:55:29--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Red_postman_butterfly_%28Heliconius_erato%29.jpg/1599px-Red_postman_butterfly_%28Heliconius_erato%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224597 (219K) [image/jpeg]
Savi

In [19]:
from google.colab import files

# https://drive.google.com/file/d/1qKiyp4r8SqUtz2ZWk3E6oZhyhl6t8lyG/view?usp=sharing
!gdown --id 1qKiyp4r8SqUtz2ZWk3E6oZhyhl6t8lyG
path_class_names = '/content/class_names_restnet_leeds_butterfly.pkl'


# https://drive.google.com/file/d/1Ep2YWU4M-yVkF7AFP3aD1sVhuriIDzFe/view?usp=sharing
!gdown --id 1Ep2YWU4M-yVkF7AFP3aD1sVhuriIDzFe
path_model = '/content/model_state_restnet_leeds_butterfly.pth'

Downloading...
From: https://drive.google.com/uc?id=1qKiyp4r8SqUtz2ZWk3E6oZhyhl6t8lyG
To: /content/class_names_restnet_leeds_butterfly.pkl
100% 247/247 [00:00<00:00, 180kB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ep2YWU4M-yVkF7AFP3aD1sVhuriIDzFe
To: /content/model_state_restnet_leeds_butterfly.pth
100% 44.8M/44.8M [00:00<00:00, 209MB/s]


In [20]:
class_names = pickle.load(open(path_class_names, "rb"))
labels = class_names

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

In [23]:
model_ft.load_state_dict(copy.deepcopy(torch.load(path_model,device)))

<All keys matched successfully>

In [24]:
def predict(inp):
  inp = Image.fromarray(inp.astype('uint8'), 'RGB')
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(model_ft(inp)[0], dim=0)
  return {labels[i]: float(prediction[i]) for i in range(len(class_names))}

In [39]:
inputs = gr.inputs.Image(label="Input")
outputs = gr.outputs.Label(num_top_classes=3)
gr.Interface(fn=predict, inputs=inputs, outputs=outputs, title="butterfly classification demo", theme="dark-peach", examples=example_inp).launch();

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://11530.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)
